# Behavioral Design Patterns

Behavioral Design Patterns focus on how objects interact with each other and emphasize communication and responsibility sharing between them. Key patterns are:

- <b>Chain of Responsibility:</b> Pass requests along a chain.
- <b>Command:</b> Encapsulate requests.
- <b>Interpreter:</b> Evaluate expressions.
- <b>Iterator:</b> Access collection elements.
- <b>Mediator:</b> Centralize communication.
- <b>Memento:</b> Save and restore state.
- <b>Observer:</b> Notify dependents.
- <b>State:</b> Change behavior with state.
- <b>Strategy:</b> Interchangeable algorithms.
- <b>Template Method:</b> Define steps in a base class

### 1. Chain of Responsibility Pattern

- <b>Purpose:</b> Allows passing a request along a chain of handlers, where each handler can either process the request or pass it to the next handler in the chain.

- <b>Uses:</b> Useful when you want to decouple the sender of a request from its receivers and allow multiple handlers to process a request independently.

In [3]:
class Handler:

    def __init__(self, successor=None):
        self.successor = successor

    def handle(self, request):
        """Process request or pass it to the next handler."""
        if self.successor:
            return self.successor.handle(request)
        return None


class ConcreteHandler(Handler):

    def __init__(self, condition, action, successor=None):
        super().__init__(successor)
        self.condition = condition
        self.action = action

    def handle(self, request):
        """Handle request if condition matches, otherwise pass to next handler."""
        if self.condition(request):
            return self.action(request)

        if self.successor:
            return self.successor.handle(request)
        return None


# Example usage:
handler_chain = ConcreteHandler(
    condition=lambda x: x < 10, 
    action=lambda x: f"Handler1 processed {x}",
    successor=ConcreteHandler(
        condition=lambda x: 10 <= x < 20, 
        action=lambda x: f"Handler2 processed {x}"
    )
)

print(handler_chain.handle(15))  # Output: Handler2 processed 15


Handler2 processed 15


### 2. Command Pattern

- <b>Purpose:</b> Encapsulates a request as an object, thereby allowing parameterization of clients with different requests, queuing of requests, and logging the requests.

- <b>Uses:</b> Useful when you need to decouple the sender and receiver of a request, and especially when you want to support undo/redo functionality or store requests for later execution.

In [5]:
class Command:

    def execute(self):
        """Execute the command."""
        pass


class Light:

    def on(self):
        return "Light is ON"

    def off(self):
        return "Light is OFF"


class LightOnCommand(Command):

    def __init__(self, light: Light):
        self.light = light

    def execute(self):
        return self.light.on()


class LightOffCommand(Command):

    def __init__(self, light: Light):
        self.light = light

    def execute(self):
        return self.light.off()

    
class RemoteControl:

    def __init__(self):
        self.command = None

    def set_command(self, command: Command):
        self.command = command

    def press_button(self):
        return self.command.execute()


# Example usage:
light = Light()
light_on = LightOnCommand(light)
light_off = LightOffCommand(light)

remote = RemoteControl()

remote.set_command(light_on)
print(remote.press_button())  # Output: Light is ON

remote.set_command(light_off)
print(remote.press_button())  # Output: Light is OFF


Light is ON
Light is OFF


### 3. Interpreter Pattern

- <b>Purpose:</b> Defines a grammar for interpreting sentences in a language and provides an interpreter to evaluate expressions defined in the grammar.

- <b>Uses:</b> Useful when you need to interpret or evaluate expressions in a custom language or domain-specific language (DSL), such as for parsing arithmetic expressions or simple query languages.

In [7]:
class Expression:

    def interpret(self, context):
        pass


# Terminal Expressions
class Number(Expression):

    def __init__(self, value):
        self.value = value

    def interpret(self, context):
        return self.value


# Non-terminal Expression
class Add(Expression):

    def __init__(self, left, right):
        self.left = left
        self.right = right

    def interpret(self, context):
        return self.left.interpret(context) + self.right.interpret(context)


class Subtract(Expression):

    def __init__(self, left, right):
        self.left = left
        self.right = right

    def interpret(self, context):
        return self.left.interpret(context) - self.right.interpret(context)


# Example usage: Expression "5 + 3 - 2"
context = {}

# Build the expression tree
expr = Subtract(Add(Number(5), Number(3)), Number(2))

# Interpret and evaluate the expression
print(f"Result: {expr.interpret(context)}")  # Output: Result: 6


Result: 6


### 4. Iterator Pattern

- <b>Purpose:</b> Provides a way to access the elements of an aggregate object sequentially without exposing its underlying representation.

- <b>Uses:</b> Useful when you want to traverse a collection (like lists, sets) without exposing the internal structure and when you need to provide a standard way of iterating over various types of collections.

In [9]:
# Iterator Interface
class Iterator:

    def __init__(self, collection):
        self.collection = collection
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.collection):
            result = self.collection[self.index]
            self.index += 1
            return result
        else:
            raise StopIteration


# Aggregate Interface
class IterableCollection:

    def __init__(self, items):
        self.items = items

    def get_iterator(self):
        return Iterator(self.items)


# Example usage
items = IterableCollection([1, 2, 3, 4, 5])
iterator = items.get_iterator()

for item in iterator:
    print(item)  # Output: 1, 2, 3, 4, 5


1
2
3
4
5


### 5. Mediator Pattern

- <b>Purpose:</b> Defines an object that centralizes communication between multiple objects, preventing them from referring to each other explicitly, thus reducing dependencies.

- <b>Uses:</b> Useful when you have a system of objects that need to communicate with each other, but you want to avoid tight coupling and direct interactions between them.

In [11]:
# Mediator Interface
class Mediator:

    def send(self, message, colleague):
        pass

    
# Concrete Mediator
class ConcreteMediator(Mediator):

    def __init__(self):
        self.colleague1 = None
        self.colleague2 = None

    def send(self, message, colleague):
        if colleague == self.colleague1:
            self.colleague2.notify(message)
        elif colleague == self.colleague2:
            self.colleague1.notify(message)

    def set_colleague1(self, colleague):
        self.colleague1 = colleague

    def set_colleague2(self, colleague):
        self.colleague2 = colleague


# Colleague Interface
class Colleague:

    def __init__(self, mediator):
        self.mediator = mediator

    def send(self, message):
        self.mediator.send(message, self)

    def notify(self, message):
        print(f"Received message: {message}")


# Example usage
mediator = ConcreteMediator()

colleague1 = Colleague(mediator)
colleague2 = Colleague(mediator)

mediator.set_colleague1(colleague1)
mediator.set_colleague2(colleague2)

colleague1.send("Hello from colleague1!")
colleague2.send("Hello from colleague2!")


Received message: Hello from colleague1!
Received message: Hello from colleague2!


### 6. Memento Pattern

- <b>Purpose:</b> Allows capturing and externalizing an object's state so that it can be restored later without violating encapsulation.

- <b>Uses:</b> Useful when you need to provide the ability to restore an object to a previous state, such as in undo/redo functionality or storing snapshots of object states.

In [13]:
# Memento Class
class Memento:

    def __init__(self, state):
        self.state = state


# Originator Class
class Originator:

    def __init__(self, state):
        self.state = state

    def set_state(self, state):
        print(f"Setting state to: {state}")
        self.state = state

    def get_state(self):
        return self.state

    def save_to_memento(self):
        return Memento(self.state)

    def restore_from_memento(self, memento):
        self.state = memento.state
        print(f"State restored to: {self.state}")


# Caretaker Class
class Caretaker:
    
    def __init__(self):
        self.mementos = []

    def add_memento(self, memento):
        self.mementos.append(memento)

    def get_memento(self, index):
        return self.mementos[index]


# Example usage
originator = Originator("State 1")
caretaker = Caretaker()

# Save state
caretaker.add_memento(originator.save_to_memento())

# Change state
originator.set_state("State 2")
caretaker.add_memento(originator.save_to_memento())

# Restore previous state
originator.restore_from_memento(caretaker.get_memento(0))


Setting state to: State 2
State restored to: State 1


### 7. Observer Pattern

- <b>Purpose:</b> Defines a one-to-many dependency between objects, where a change in one object triggers updates in dependent objects without tightly coupling them.

- <b>Uses:</b> Useful in scenarios where multiple objects need to be notified and updated automatically when a change occurs in another object, such as in event handling or real-time data updates.

In [15]:
from abc import ABC, abstractmethod


# Observer Interface
class Observer(ABC):
    
    @abstractmethod
    def update(self, message):
        pass


# Concrete Observer
class ConcreteObserver(Observer):

    def __init__(self, name):
        self.name = name

    def update(self, message):
        print(f"{self.name} received message: {message}")


# Subject Interface
class Subject(ABC):

    @abstractmethod
    def add_observer(self, observer: Observer):
        pass

    @abstractmethod
    def remove_observer(self, observer: Observer):
        pass

    @abstractmethod
    def notify_observers(self):
        pass


# Concrete Subject
class ConcreteSubject(Subject):

    def __init__(self):
        self._observers = []
        self._state = None

    def add_observer(self, observer: Observer):
        self._observers.append(observer)

    def remove_observer(self, observer: Observer):
        self._observers.remove(observer)

    def notify_observers(self):
        for observer in self._observers:
            observer.update(self._state)

    def set_state(self, state):
        print(f"State changed to: {state}")
        self._state = state
        self.notify_observers()


# Example usage
subject = ConcreteSubject()
observer1 = ConcreteObserver("Observer 1")
observer2 = ConcreteObserver("Observer 2")

subject.add_observer(observer1)
subject.add_observer(observer2)

subject.set_state("New State 1")  # Both observers will be notified
subject.set_state("New State 2")  # Both observers will be notified


State changed to: New State 1
Observer 1 received message: New State 1
Observer 2 received message: New State 1
State changed to: New State 2
Observer 1 received message: New State 2
Observer 2 received message: New State 2


### 8. State Pattern

- <b>Purpose:</b> Allows an object to alter its behavior when its internal state changes, treating state-specific behaviors as separate classes.

- <b>Uses:</b> Useful when an object's behavior depends on its state, and it must change its behavior dynamically at runtime based on that state.

In [17]:
from abc import ABC, abstractmethod


# State Interface
class State(ABC):

    @abstractmethod
    def handle(self):
        pass


# Concrete States
class ConcreteStateA(State):

    def handle(self):
        print("Handling state A")


class ConcreteStateB(State):

    def handle(self):
        print("Handling state B")


# Context class that changes its state
class Context:

    def __init__(self, state: State):
        self._state = state

    def set_state(self, state: State):
        print(f"State changed to {state.__class__.__name__}")
        self._state = state

    def request(self):
        self._state.handle()


# Example usage
context = Context(ConcreteStateA())

context.request()  # Output: Handling state A
context.set_state(ConcreteStateB())
context.request()  # Output: Handling state B


Handling state A
State changed to ConcreteStateB
Handling state B


### 9. Strategy Pattern

- <b>Purpose:</b> Defines a family of algorithms, encapsulates each one, and makes them interchangeable, allowing the algorithm to be selected at runtime.

- <b>Uses:</b> Useful when you have multiple algorithms for a specific task and want to allow the client to choose the most suitable one without modifying the context class.

In [19]:
from abc import ABC, abstractmethod


# Strategy Interface
class Strategy(ABC):

    @abstractmethod
    def execute(self, a, b):
        pass


# Concrete Strategies
class ConcreteStrategyAdd(Strategy):

    def execute(self, a, b):
        return a + b


class ConcreteStrategySubtract(Strategy):

    def execute(self, a, b):
        return a - b


class ConcreteStrategyMultiply(Strategy):

    def execute(self, a, b):
        return a * b


# Context class that uses a Strategy
class Context:

    def __init__(self, strategy: Strategy):
        self._strategy = strategy

    def set_strategy(self, strategy: Strategy):
        self._strategy = strategy

    def execute_strategy(self, a, b):
        return self._strategy.execute(a, b)


# Example usage
context = Context(ConcreteStrategyAdd())
print("Addition:", context.execute_strategy(5, 3))  # Output: 8

context.set_strategy(ConcreteStrategySubtract())
print("Subtraction:", context.execute_strategy(5, 3))  # Output: 2

context.set_strategy(ConcreteStrategyMultiply())
print("Multiplication:", context.execute_strategy(5, 3))  # Output: 15


Addition: 8
Subtraction: 2
Multiplication: 15


### 10. Template Method Pattern

- <b>Purpose:</b> Defines the structure of an algorithm in the superclass, but lets subclasses override specific steps of the algorithm without changing its structure.

- <b>Uses:</b> Useful when you have a common algorithm with some customizable steps, allowing the algorithm to be reused while still allowing flexibility for customization.

In [21]:
from abc import ABC, abstractmethod


# Abstract class defining the template method
class AbstractClass(ABC):

    def template_method(self):
        self.step_one()
        self.step_two()
        self.step_three()

    @abstractmethod
    def step_one(self):
        pass

    @abstractmethod
    def step_two(self):
        pass

    @abstractmethod
    def step_three(self):
        pass


# Concrete classes implementing the abstract steps
class ConcreteClassA(AbstractClass):

    def step_one(self):
        print("ConcreteClassA: Step One")

    def step_two(self):
        print("ConcreteClassA: Step Two")

    def step_three(self):
        print("ConcreteClassA: Step Three")


class ConcreteClassB(AbstractClass):

    def step_one(self):
        print("ConcreteClassB: Step One")

    def step_two(self):
        print("ConcreteClassB: Step Two")

    def step_three(self):
        print("ConcreteClassB: Step Three")


# Example usage
concreteA = ConcreteClassA()
concreteA.template_method()

concreteB = ConcreteClassB()
concreteB.template_method()


ConcreteClassA: Step One
ConcreteClassA: Step Two
ConcreteClassA: Step Three
ConcreteClassB: Step One
ConcreteClassB: Step Two
ConcreteClassB: Step Three
